# Hot Wheels Data Processing
This notebook processes Hot Wheels data and loads it into a SQLite database.

## 1. Setup and Data Loading

In [9]:
import pandas as pd
import numpy as np
import uuid
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()

POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_HOSTNAME = os.getenv("POSTGRES_HOSTNAME")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")

# Load the data
df = pd.read_json("hotwheels.jsonl", lines=True, dtype={"Collector #": "Int64"})

## 2. Data Cleaning

In [10]:
# Remove rows with missing model names and drop unnecessary columns
df = df.dropna(subset=['Model Name'])
df = df.drop(columns=["Card Variant", "Mainline", "ID"])

## 3. Column Standardization

In [11]:
# Rename columns to snake_case format
column_mapping = {
    'Model Name': 'model_name', 
    'Image URL': 'image_url',
    'Collector #': 'collector_number',
    'Series #': 'series_number',
    'Release Year': 'release_year',
    'Series': 'series',
    'Color': 'color',
    'Tampo': 'tampo',
    'Wheel Type': 'wheel_type',
    'Base Type': 'base_type',
    'Base Color': 'base_color',
    'Window Color': 'window_color',
    'Interior Color': 'interior_color',
    'Toy #': 'toy_number',
    'Assortment #': 'assortment_number',
    'Scale': 'scale',
    'Country': 'country',
    'Base Codes': 'base_codes',
    'Case Number': 'case_number',
    'Notes': 'notes',
    'Treasure Hunt': 'treasure_hunt'
}
df = df.rename(columns=column_mapping)

## 4. Feature Engineering

In [12]:
def extract_treasure_hunt_year(text):
    if pd.isna(text):
        return np.nan
    return int(text.split(' ')[0]) if 'Treasure Hunts' in text and 'Super' not in text else np.nan

def extract_super_treasure_hunt_year(text):
    if pd.isna(text):
        return np.nan
    return int(text.split(' ')[0]) if 'Super Treasure Hunts' in text else np.nan

# Create treasure hunt features
df['treasure_hunt_year'] = df['treasure_hunt'].apply(extract_treasure_hunt_year)
df['super_treasure_hunt_year'] = df['treasure_hunt'].apply(extract_super_treasure_hunt_year)
df = df.drop(columns=["treasure_hunt"])

# Add unique identifier
df['id'] = [uuid.uuid4() for _ in range(len(df))]

## 5. Data Type Conversion

In [13]:
# Convert columns to appropriate data types
df["super_treasure_hunt_year"] = df["super_treasure_hunt_year"].astype("Int64")
df["treasure_hunt_year"] = df["treasure_hunt_year"].astype("Int64")
df["release_year"] = pd.to_numeric(df["release_year"], errors='coerce').astype("Int64")

In [14]:
df[['model_name', 'image_url', 'series', 'release_year']].sample(11)

,model_name,image_url,series,release_year
6740,Classic '36 Ford Coupe,NaN,Hot Wheels,1969
47288,MS-T Suzuka,https://images.collecthw.com/c3eced29-0133-4a7...,HW 2001,2001
3571,Nitro Scorcher,https://images.collecthw.com/5ea8cb29-fc93-48a...,2007 New Models,2007
4313,Way 2 Fast,https://images.collecthw.com/cbb43eee-31d2-40a...,Pin Hedz,2005
30345,Maximum Leeway,https://images.collecthw.com/4fd2f788-66a0-41b...,HW Games,2016
33826,Lamborghini Sesto Elemento,https://images.collecthw.com/e3f09398-71eb-4e6...,HW Exotics,2020
40194,2006 Dodge Viper Coupe,https://images.collecthw.com/72c2bc06-bc52-45d...,2006 First Editions,2006
31543,'17 Acura NSX,https://images.collecthw.com/2d09b093-e1f5-439...,HW Exotics,2019
16772,Side Draft,https://images.collecthw.com/a6f5c025-a25a-42a...,Multipack Exclusive,2012
53805,'72 Plymouth Cuda FC,https://images.collecthw.com/f3d00bab-9fae-4c1...,NFT Garage Series 8,2024


In [15]:
df["visit_count"] = 0
df.to_csv("hotwheels.csv", index=False)

# Create a code that reads the CSV file and writes the data to a PostgreSQL database
df = pd.read_csv("hotwheels.csv")
engine = create_engine(f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOSTNAME}:{POSTGRES_PORT}/{POSTGRES_DB}")
df.to_sql("hotwheels", engine, if_exists="append", index=False)

13

## 6. Database Export

In [16]:
""" DATABASE_URL = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOSTNAME}:{POSTGRES_PORT}/{POSTGRES_DB}"
engine = create_engine(DATABASE_URL)

df.to_sql('hotwheels', engine, if_exists='append', index=False) """

' DATABASE_URL = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOSTNAME}:{POSTGRES_PORT}/{POSTGRES_DB}"\nengine = create_engine(DATABASE_URL)\n\ndf.to_sql(\'hotwheels\', engine, if_exists=\'append\', index=False) '